In [1]:
import requests
import re 
import os.path
from os import path
import bs4
from bs4 import BeautifulSoup
# import couchdb

# I scripted this in urgency, So code quality or the logic isn't great.
# This is mostly a hack. 

# To RUN
# Clone the project
# Set up a virual environment 
# Install everything in requirements
# setup pushover_api_token, pushover_user_key and covid_db_full_url in env
# needs couchdb
# run this by calling `python3 scraper.py`
# You can also schedule it using the digdag
states = {}
states["Andhra Pradesh"]="AP"
states["Arunachal Pradesh"]="AR"
states["Assam"]="AS"
states["Bihar"]="BR"
states["Chattisgarh"]="CT"
states["Chhattisgarh"]="CT"
states["Goa"]="GA"
states["Gujarat"]="GJ"
states["Haryana"]="HR"
states["Himachal Pradesh"]="HP"
states["Jharkhand"]="JH"
states["Karnataka"]="KA"
states["Kerala"]="KL"
states["Madhya Pradesh"]="MP"
states["Maharashtra"]="MH"
states["Manipur"]="MN"
states["Meghalaya"]="ML"
states["Mizoram"]="MZ"
states["Nagaland"]="NL"
states["Odisha"]="OR"
states["Punjab"]="PB"
states["Rajasthan"]="RJ"
states["Sikkim"]="SK"
states["Tamil Nadu"]="TN"
states["Telengana"]="TG"
states["Tripura"]="TR"
states["Uttarakhand"]="UT"
states["Uttar Pradesh"]="UP"
states["West Bengal"]="WB"
states["Andaman and Nicobar Islands"]="AN"
states["Chandigarh"]="CH"
states["Dadra and Nagar Haveli"]="DN"
states["Daman and Diu"]="DD"
states["Delhi"]="DL"
states["Jammu and Kashmir"]="JK"
states["Ladakh"]="LA"
states["Lakshadweep"]="LD"
states["Pondicherry"]="PY"
states["Puducherry"]="PY"

In [2]:
def getFileName(t):
	t = t.replace("as on ","")
	t = t.replace(".","-")
	t = t.replace(" ","-")
	t = t.replace(":","-")
	t = t+".html"
	return t

# This is not a great way of doing this. This is just a hack
# I foculd have used standard date formatter!! 
def getFormattedDate(t):
	x = t.replace("as on ","", 1)
	parts = x.split(" at ")

	date_parts = (parts[0]).split(".")
	day = date_parts[0]
	if len(day) == 1:
		day = "0"+day

	month = date_parts[1]
	if len(month) == 1:
		month = "0"+month

	year = date_parts[2]
	time_parts = (parts[1]).split(" ")
	am_pm = time_parts[1]
	hour = ((time_parts[0]).split(":"))[0]

	if am_pm == "PM":
		hour = str(int(hour)+12)
	if len(hour) == 1:
		hour = "0"+hour

	minute = ((time_parts[0]).split(":"))[1]
	if len(minute) == 1:
		minute = "0"+minute

	std_format = "2020-{0}-{1}T{2}:{3}:00.00+05:30".format(month, day,hour,minute)
	return str(std_format)

def getContents():
	url = "https://www.mohfw.gov.in/"
	r = requests.get(url)	
	txt = ""
	if r.status_code == 200:
		txt = r.text
		return txt


In [3]:
# database = couch.create(couchdb_db_name) # newly created
# >>> db = couch['mydb'] # existing

In [5]:
# title = "Started"
# message = "started \n"
# couchdb_db_name = "covid19"
# couch = couchdb.Server(covid_db_full_url)
# database = couch[couchdb_db_name]

txt = getContents()

date_pattern = "as on \d\d.\d\d.2020 at \d\d:\d\d [A,P]M"
x = re.findall(date_pattern, txt)
extracted_date_text = x[0]

full_date_text =  getFormattedDate(extracted_date_text)
# full_file_name =  archive_folder_path.format(getFileName(extracted_date_text))

print(extracted_date_text)
# print(full_file_name)
print(full_date_text)

import pandas as pd

data = {}
data["_id"] = []
data["state"] = []
data["report_time"] = []
data["confirmed_india"] =  []
data["confirmed_foreign"] = []
data["cured"] = []
data["death"] = []
#get to parsing
soup = BeautifulSoup(txt, 'html.parser')
tables = soup.find_all('tbody')
coviddata=[]
if len(tables) > 0:
    table = tables[-1]
    #print(table)
    first_row = False
    for tr in list(table.children):
        if isinstance(tr, bs4.element.Tag): 
            if first_row:
                print("FIRST ROW")
                first_row = False
                continue

            tds = list(tr.children)

            if "Total number of confirmed cases in India" == tds[1].get_text():
                continue


            report_time = full_date_text
            state = (tds[3]).get_text()
            state = state.replace("Union Territory of ","")
            state = state.strip()
            state_code = ""
            if state in states :
                state_code = (states[state]).lower()
            else:
                sendMessage("ERROR WRONG STATE",  "Not found: {0}".format(state))
                print("------> Wrong state {0}".format(state))
#                 return 0

            _id =  report_time+"|"+state_code

#             data = {}
            data["_id"].append(_id)
            data["state"].append(state_code)
            data["report_time"].append(report_time)
            data["confirmed_india"].append(int( (tds[5]).get_text() ))
            data["confirmed_foreign"].append(int( (tds[7]).get_text() ))
            data["cured"].append(int( (tds[9]).get_text() ))
            data["death"].append(int( (tds[11]).get_text() ))

else:
    print("No Table found")
# print(data)

# tables[-1]
# data
df=pd.DataFrame.from_dict(data)
# df
excelfile=report_time[0:10]+"_"+report_time[11:12]+"h"+report_time[14:16]+"m.csv"
# excelfile
# df.to_csv(excelfile,index=False)

jdf=pd.read_csv('covid_india.csv')
# jdf=jdf.drop(["_rev"], axis = 1)
if (df["_id"][0] not in jdf["_id"].values):
    jdf=jdf.append(df)
    jdf.to_csv('covid_india.csv',index=False)
    print("Your file sucessfully updated")
else:
    print("data already exists")



as on 23.03.2020 at 08:15 PM
2020-03-23T20:15:00.00+05:30
data already exists


In [114]:
# jsondf=pd.read_json("..\..\data\mohfw1.json")
# # pd.read_json(jsondf['rows'][0])
# ms="{'row 1':"+str(jsondf["value"][0])+','
# ms +="'row 2':"+str(jsondf["value"][1])+'}'
# ms

# # list(jsondf["value"][0].keys())

# _='{"row 1":{"col 1":"a","col 2":"b"},"row 2":{"col 1":"c","col 2":"d"}}'
# # pd.read_json(ms, orient='index')
# import json    # or `import simplejson as json` if on Python < 2.6

# # json_string = u'{ "id":"123456789"}'
# jdf=pd.DataFrame([jsondf["value"][0].values()],columns=list(jsondf["value"][0].keys()))
# for i in range(1,len(jsondf["value"])):
#     jdf=jdf.append(pd.DataFrame([jsondf["value"][i].values()],columns=list(jsondf["value"][i].keys())))
# jdf


,_id,_rev,report_time,state,confirmed_india,confirmed_foreign,cured,death
0,2020-03-10T12:00:00.00+05:30|dl,1-53c2cb501eb15734ac4f568d27f5881d,2020-03-10T12:00:00.00+05:30,dl,4,0,0,0
0,2020-03-10T12:00:00.00+05:30|hr,1-bb5ef04e46ac504dcaebf598fb7e2f04,2020-03-10T12:00:00.00+05:30,hr,0,14,0,0
0,2020-03-10T12:00:00.00+05:30|jk,1-a04baaf575c349c23d5ab4e62ceccde5,2020-03-10T12:00:00.00+05:30,jk,1,0,0,0
0,2020-03-10T12:00:00.00+05:30|ka,1-531e3fcb0d398b7379f811161bf788c2,2020-03-10T12:00:00.00+05:30,ka,1,0,0,0
0,2020-03-10T12:00:00.00+05:30|kl,1-3573aec5f84131105a2ab789c2a6e0ca,2020-03-10T12:00:00.00+05:30,kl,9,0,0,0
...,...,...,...,...,...,...,...,...
0,2020-03-21T16:45:00.00+05:30|tg,1-05cac8e8242b575de950fc35b1f79316,2020-03-21T16:45:00.00+05:30,tg,10,11,1,0
0,2020-03-21T16:45:00.00+05:30|tn,1-6cc3305c24d02b5664507573f0a4f389,2020-03-21T16:45:00.00+05:30,tn,3,0,1,0
0,2020-03-21T16:45:00.00+05:30|up,1-2ace0e2af95965980e5b8ddcbe1c2cee,2020-03-21T16:45:00.00+05:30,up,23,1,9,0
0,2020-03-21T16:45:00.00+05:30|ut,1-66a699ef15d4174ef7405b2cc44e10f9,2020-03-21T16:45:00.00+05:30,ut,3,0,0,0
